In [ ]:
# PAL: Program-Aided Language Modeling

In [ ]:
%%capture
!pip install OpenAI

In [ ]:
from openai import OpenAI
from datetime import datetime
from dateutil.relativedelta import relativedelta
import os
from google.colab import userdata

In [ ]:
# API configuration
client = OpenAI(
    api_key=userdata.get("DEEPSEEK_API_KEY"),
    base_url="https://api.deepseek.com/v1"
)

In [ ]:
# get completion
def get_completion(messages, temperature=0, max_tokens=1000):
    response = client.chat.completions.create(
        model="deepseek-chat",
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens
    )
    return response.choices[0].message.content

In [ ]:
get_completion(messages=[
          {"role": "system", "content": "You are a helpful assistant."},
          {"role": "user", "content": "Hello, how are you?"}
      ],)

"Hello! I'm doing great, thank you for asking. How are you today? 😊"

In [ ]:
user_question = "Today is 12 September 2025. I was born exactly 48 years ago. What is the date I was born in MM/DD/YYYY?."

DATE_UNDERSTANDING_PROMPT = """
# Q: 2015 is coming in 36 hours. What is the date one week from today in MM/DD/YYYY?
# If 2015 is coming in 36 hours, then today is 36 hours before.
today = datetime(2015, 1, 1) - relativedelta(hours=36)
# One week from today,
one_week_from_today = today + relativedelta(weeks=1)
# The answer formatted with %m/%d/%Y is
answer = one_week_from_today.strftime('%m/%d/%Y')

# Q: The first day of 2019 is a Tuesday, and today is the first Monday of 2019. What is the date today in MM/DD/YYYY?
# If the first day of 2019 is a Tuesday, and today is the first Monday of 2019, then today is 6 days later.
today = datetime(2019, 1, 1) + relativedelta(days=6)
# The answer formatted with %m/%d/%Y is
answer = today.strftime('%m/%d/%Y')

# Q: The concert was scheduled to be on 06/01/1943, but was delayed by one day to today. What is the date 10 days ago in MM/DD/YYYY?
# If the concert was scheduled to be on 06/01/1943, but was delayed by one day to today, then today is one day later.
today = datetime(1943, 6, 1) + relativedelta(days=1)
# 10 days ago,
ten_days_ago = today - relativedelta(days=10)
# The answer formatted with %m/%d/%Y is
answer = ten_days_ago.strftime('%m/%d/%Y')

# Q: It is 4/19/1969 today. What is the date 24 hours later in MM/DD/YYYY?
# It is 4/19/1969 today.
today = datetime(1969, 4, 19)
# 24 hours later,
later = today + relativedelta(hours=24)
# The answer formatted with %m/%d/%Y is
answer = today.strftime('%m/%d/%Y')

# Q: Jane thought today is 3/11/2002, but today is in fact Mar 12, which is 1 day later. What is the date 24 hours later in MM/DD/YYYY?
# If Jane thought today is 3/11/2002, but today is in fact Mar 12, then today is 3/1/2002.
today = datetime(2002, 3, 12)
# 24 hours later,
later = today + relativedelta(hours=24)
# The answer formatted with %m/%d/%Y is
answer = later.strftime('%m/%d/%Y')

# Q: Jane was born on the last day of Feburary in 2001. Today is her 16-year-old birthday. What is the date yesterday in MM/DD/YYYY?
# If Jane was born on the last day of Feburary in 2001 and today is her 16-year-old birthday, then today is 16 years later.
today = datetime(2001, 2, 28) + relativedelta(years=16)
# Yesterday,
yesterday = today - relativedelta(days=1)
# The answer formatted with %m/%d/%Y is
answer = yesterday.strftime('%m/%d/%Y')

# Q: {question}
""".strip() + '\n'

messages = [
    {"role": "system", "content": "You are a helpful assistant. Use the same format as the examples delimited by <examples></examples>. Don't use python output."},
    {"role": "user", "content": DATE_UNDERSTANDING_PROMPT.format(question=user_question)}
]

In [ ]:
llm_out = get_completion(messages)
print(llm_out)

# If today is 12 September 2025 and I was born exactly 48 years ago, then the birth date is 48 years earlier.
birth_date = datetime(2025, 9, 12) - relativedelta(years=48)
# The answer formatted with %m/%d/%Y is
answer = birth_date.strftime('%m/%d/%Y')


In [ ]:
exec(llm_out)
print(answer)

09/12/1977
